In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import paper_vrp_mini as research

# for plotting
from paper_vrp_mini.visuals import *
set_visuals()


In [ ]:
# display plots inline
%matplotlib inline

In [ ]:
# ResearchData provides the database for stuff related to the paper;
# initialize it with a correct path
hangar = research.ResearchData("../vrp_paper_hangar_mini.h5")

# Example 1: covariance matrix from a row of variances

In [ ]:
# parameters
horizon_s = "3m"
counter_currency = "usd"
row_idx = "2015-01-14"

In [ ]:
# fetch MFIV from the database, take one row
mfiv = hangar.get("mfiv/m" + horizon_s)
mfiv_row = mfiv.loc[row_idx]

In [ ]:
# calculate
mficov = research.fx_covmat_from_variances(mfiv_row, 
                                           counter_currency)

In [ ]:
# plot as heatmap
fig, ax = plt.subplots(figsize=visuals.figsize_2, facecolor='w')
sns.heatmap(mficov.pow(1/2)*100, 
            ax=ax, annot=True, fmt=".2f", square=False, 
            cmap=visuals.heatmap_cmap)

# Example 2: from covariance matrix to variance of a currency portfolio

In [ ]:
# define weights of currencies
portfolio = pd.Series({"aud": 1/3, "nzd": 1/3, "cad": 1/3, 
                       "jpy": -1/3, "chf": -1/3, "eur": -1/3})

# reindex to follow the layout of the covariance matrix
portfolio = portfolio.reindex(mficov.columns, fill_value=0.0)

# convert to CovarianceDataFrame for easier manipulations
mficov_c = research.CovarianceDataFrame(mficov)

# calculate variance as a quadratic form
portfolio_var = mficov_c.quadratic_form(other=portfolio,
                                        trim=True)

In [ ]:
# plot the portfolio variance along individual variances 
# (i.e. variances of appreciation rates vs. USD)
to_plot = mficov_c.get_variances()\
    .assign(carry=portfolio_var)\
    .iloc[0]\
    .pow(1/2) * 100
    
fig, ax = plt.subplots(figsize=visuals.figsize_1, facecolor='w')
to_plot.plot(ax=ax, linestyle="none", marker='o')
ax.set_xticks(range(len(to_plot)))
ax.set_xticklabels(to_plot.index)

# Figure 3: Eigendecomposition of variance swap payoff structure matrices

In [ ]:
# set up research environment:
currencies = ["aud", "cad", "chf", "eur", "gbp", "jpy", "nzd", "usd"]
counter_currency = "usd"
start_dt, end_dt = "2009-01-14", "2018-06-30"

env = research.ResearchUniverse(
    hangar, currencies, counter_currency, start_dt, end_dt
)

In [ ]:
# functions to perform eigendecomposition
def get_eigvectors(vcv):
    """Calculate eigenvectors.
    Sorts eigenvectors by the size of their eigenvalues.
    
    Returns
    -------
    res : pandas.DataFrame
        matrix with eigenvectors in rows, columned just like `vcv`
    """
    eigval, eigvec = np.linalg.eig(vcv.values)
    arg_sort = np.argsort(np.real(eigval))
    
    loadings = pd.DataFrame(
        data=eigvec[:, arg_sort],
        index=vcv.columns,
        columns=range(1, len(eigval) + 1))

    res = loadings.T

    return res

def get_eigvalues(vcv):
    """Calculate eigenvalues.
    Sorts the eigenvalues.
    
    Returns
    -------
    res : pandas.Series
        of eigenvalues, sorted
    """
    eigval = np.linalg.eigvals(vcv.values)

    res = pd.Series(
        np.sort(eigval),
        index=range(1, len(eigval) + 1))

    return res

In [ ]:
# fetch mficovariance and realized covariance
h = "3m"
mficov = env.get_mficov(h)
rcov = env.get_rcov(h, look="forward")

In [ ]:
# synthetic variance swap payoffs structure matrices
vs_p = rcov - mficov
vs_p = vs_p.dropna(how="any")

In [ ]:
# calculate loadings and variance risk premia of corresponding
#   portfolios
eigvec = vs_p.groupby_time().apply(get_eigvectors)
eigval = vs_p.groupby_time().apply(get_eigvalues)

In [ ]:
# make loading of AUD be always positive
#   (switching the sign of any eigenvector leaves 
#   vr premium unchanged)
idx_aud_neg = eigvec.loc[:, "aud"] < 0
eigvec.loc[idx_aud_neg, :] = eigvec.loc[idx_aud_neg, :]*-1

In [ ]:
# calculate average values of loadings and vr premia
eigvec_mu = eigvec\
    .unstack(level=1)\
    .mean()\
    .unstack(level=0)
eigval_mu = eigval.mean().to_frame()

In [ ]:
# plot (requires seaborn)
fig, ax = plt.subplots(1, 2, figsize=figsize_1,
                       gridspec_kw={'width_ratios': [10, 1]},
                       sharex=False, sharey=False)

# plot eigvectors
sns.heatmap(eigvec_mu, ax=ax[0], cmap=heatmap_cmap, 
            square=False, linewidths=0.5, 
            center=0, vmin=-0.5, vmax=0.5,
            annot=True, fmt=".2f", annot_kws={"fontsize": 9},
            cbar_kws={'label': 'portfolio weight',
                      "format": "%.1f"})

# plot eigvalues (vr premium)
sns.heatmap(eigval_mu * 1e04, ax=ax[1], cmap=heatmap_cmap,
            square=False, linewidths=0.5,
            center=0, 
            annot=True, fmt=".0f", annot_kws={"fontsize": 9},
            cbar=False)

# aestetics
ax[0].set_yticklabels(ax[0].get_yticklabels(), rotation="horizontal",
                      va='center', minor=False)
ax[0].set_xlabel("", visible=False)
ax[0].set_ylabel("portfolio")
ax[1].set_xlabel("", visible=False)

ax[1].set_xticklabels([r"$vs$"], ha='center')
ax[1].yaxis.tick_right()
ax[1].yaxis.set_visible(False)
ax[1].set_yticklabels([])

fig.tight_layout()